In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import seaborn as sns
from sklearn.cluster import DBSCAN
import pickle
import time
from sklearn.linear_model import LinearRegression
import random
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import sys

In [9]:
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    
    return output

def save_data(filename, data):
    a_file = open(filename, "wb")
    pickle.dump(data, a_file)
    a_file.close()

In [10]:
train_data = load_data('Processed_Data/train_set.pkl')
test_data = load_data('Processed_Data/test_set.pkl')

In [11]:
total_samples = 0
below = 0
mean_time = []
for traj in train_data:
    datetime = traj[2]
    total_samples += len(datetime)

    time_diff = np.array([t.seconds for t in np.diff(np.array(datetime))])
    below += np.sum(time_diff < 30)
    
    mean_time.append(np.mean(time_diff))

In [12]:
below, total_samples, below/total_samples

(32200, 3540044, 0.00909593214095644)

In [13]:
np.mean(np.array(mean_time)), np.std(np.array(mean_time))

(96.75714169482498, 64.87654221075653)

In [14]:
def fit_curve(times, feature, degree = 3, delta = 30):
    z = np.polyfit(times, feature, degree)
    f = np.poly1d(z)
    
    x_new = np.linspace(times[0], times[-1], int((times[-1] + delta)/delta))
    y_new = f(x_new)
    
    return x_new, y_new

In [16]:
delta = 30
new_training_data = []
for i, traj in enumerate(train_data):
    long = np.array(traj[1])
    lat = np.array(traj[0])
    datetime = traj[2]
    
    time_diff = np.array([0] + [t.seconds for t in np.diff(np.array(datetime))])
    times = np.cumsum(time_diff)
    
    x_new_lat, y_new_lat = fit_curve(times, lat, degree = 10, delta = 30)
    x_new_long, y_new_long = fit_curve(times, long, degree = 10, delta = 30)
    
    new_training_data.append([list(y_new_lat), list(y_new_long), list(x_new_lat)])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit m

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit m

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned



In [17]:
len(new_training_data), len(train_data)

(20000, 20000)

In [18]:
sys.getsizeof(train_data)/1024, sys.getsizeof(new_training_data)/1024

(167.109375, 173.859375)

In [19]:
save_data('Processed_Data/eventime_train_set.pkl', new_training_data)

In [20]:
delta = 30
new_test_data = []
for i, traj in enumerate(test_data):
    long = np.array(traj[1])
    lat = np.array(traj[0])
    datetime = traj[2]
    
    time_diff = np.array([0] + [t.seconds for t in np.diff(np.array(datetime))])
    times = np.cumsum(time_diff)
    
    x_new_lat, y_new_lat = fit_curve(times, lat, degree = 10, delta = 30)
    x_new_long, y_new_long = fit_curve(times, long, degree = 10, delta = 30)
    
    new_test_data.append([list(y_new_lat), list(y_new_long), list(x_new_lat)])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RankWarning:

Polyfit may be poorly conditioned

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RankWarning:

Polyfit may be poorly conditioned



In [21]:
len(new_test_data), len(test_data)

(1596, 1596)

In [22]:
save_data('Processed_Data/eventime_test_set.pkl', new_test_data)

In [21]:
###analysing the data

In [22]:
lengths = []
for traj in new_training_data:
    lat = traj[0]
    lengths.append(len(lat))
    

In [25]:
np.mean(np.array(lengths)), np.std(np.array(lengths))

(598.0548, 1940.7382618470117)

In [26]:
lengths = []
for traj in train_data:
    lat = traj[0]
    lengths.append(len(lat))
    

In [27]:
np.mean(np.array(lengths)), np.std(np.array(lengths))

(177.0022, 581.8124914396046)